In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib inline

import rtbm.layers as layers
import rtbm.model as mdl

import warnings
warnings.filterwarnings('ignore')

from rtbm.costfunctions import mse, crossentropy
from rtbm import minimizer

from sklearn.metrics import confusion_matrix, classification_report
from sklearn.preprocessing import LabelBinarizer

from scipy import signal

RTBM_precision= 1e-5

Process PoolWorker-8:
Traceback (most recent call last):
Process PoolWorker-3:
Process PoolWorker-1:
Process PoolWorker-7:
  File "/usr/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
Process PoolWorker-2:
Process PoolWorker-6:
  File "/usr/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
Process PoolWorker-5:
Process PoolWorker-4:
    self.run()
    self.run()
    self.run()
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib/python2.7/multiprocessing/process.py", line 114, in run
Traceback (most recent call last):
  File "/usr/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
  File "/usr/lib/python2.7

# Example 1: Sine-Mixture with gaussian noise

In [ ]:
def func(x,n):
    return 0.5*np.sin(1*x+0.1)+0.75*np.sin(0.25*x-0.3)+n*np.random.normal(0,0.2,x.shape[2])

X = np.linspace(0, 100, 500)
X = X.reshape((1,X.shape[0]))

Y = func(X[:,None],1).reshape((1,X.shape[1]))

In [ ]:
plt.plot(X.flatten(), Y.flatten(),"ob-")

In [ ]:
M = mdl.Model()
M.add(layers.DiagExpectationUnitLayer(1,2,phase=1))
M.add(layers.DiagExpectationUnitLayer(2,2,phase=1))
M.add(layers.DiagExpectationUnitLayer(2,1,phase=1))

In [ ]:
minim = minimizer.CMA(False)
minim.train(mse(), M, X, Y, maxiter=1000)

In [ ]:
npoints = 500
test_X = (np.linspace(0, 100, npoints)).reshape((1, npoints))

plt.plot(X.flatten(), Y.flatten(),"og-", label='target')
plt.plot(test_X.flatten(), np.real(M.predict(test_X)).flatten(),"b-", label='fit')
plt.legend()

In [ ]:
M.get_layer(1)._Nout

In [ ]:
M.get_layer(1).show_activation(1,10)

In [ ]:
M = mdl.Model()

M.add(layers.DiagExpectationUnitLayer(1,2))
M.add(layers.DiagExpectationUnitLayer(2,2))
M.add(layers.DiagExpectationUnitLayer(2,1))

minim = minimizer.SGD()
minim.train(mse(), M, X, Y, lr=10, maxiter=10000, nesterov=True,momentum=0.1, batch_size=100)

In [ ]:
npoints = 500
test_X = (np.linspace(0, 100, npoints)).reshape((1, npoints))

plt.plot(X.flatten(), Y.flatten(),"og-", label='target')
plt.plot(test_X.flatten(), np.real(M.predict(test_X)).flatten(),"ob-", label='fit')
plt.legend()

# Example 2: Sine-Cosine-Mixture with trend and gaussian noise

In [ ]:
def func(x,n):
    return 0.02*x+0.5*np.sin(1*x+0.1)+0.75*np.cos(0.25*x-0.3)+n*np.random.normal(0,0.2,x.shape[2])

X = np.linspace(0, 100, 500)
X = X.reshape((1,X.shape[0]))

Y = func(X[:,None],1).reshape((1,X.shape[1]))

In [ ]:
Y = func(X[:,None],0).reshape((1,X.shape[1]))

plt.plot(X.flatten(), Y.flatten(),"-")
plt.xlabel("t")
plt.ylabel("y")
plt.savefig("sincosmix.eps", format="eps")
plt.show()


In [ ]:
Y = func(X[:,None],1).reshape((1,X.shape[1]))

plt.plot(X.flatten(), Y.flatten(),"-")
plt.xlabel("t")
plt.ylabel("y")
plt.savefig("sincosmix_noise.eps", format="eps")

plt.show()



In [ ]:
M = mdl.Model()

M.add(layers.DiagExpectationUnitLayer(1,3))
M.add(layers.DiagExpectationUnitLayer(3,3))
M.add(layers.DiagExpectationUnitLayer(3,2))
M.add(layers.DiagExpectationUnitLayer(2,1))

In [ ]:
minim = minimizer.CMA(parallel=True)
minim.train(mse(), M, X, Y, tolfun=1e-4)

In [ ]:
npoints = 500
test_X = (np.linspace(0, 100, npoints)).reshape((1, npoints))

plt.plot(X.flatten(), Y.flatten(),"og", label='train')
plt.plot(test_X.flatten(), np.real(M.predict(test_X)).flatten(),"b-", label='fit', linewidth=5)
plt.legend()
plt.xlabel("t")
plt.ylabel("y")
plt.savefig("sincosmix_10-4.eps", format="eps")
plt.show()

# Example: Intrawave modulation

In [ ]:
def func(x,n):
    return np.cos(0.01*x+0.5*np.cos(0.05*x))/(2+np.sin(0.04*x))

X = np.linspace(0, 1000, 1000)
X = X.reshape((1,X.shape[0]))

Y = func(X[:,None],1).reshape((1,X.shape[1]))

plt.plot(X.flatten(), Y.flatten(),"b-")

In [ ]:
M = mdl.Model()

M.add(layers.DiagExpectationUnitLayer(1,5, phase=1j))
M.add(layers.DiagExpectationUnitLayer(5,5, phase=1j))
M.add(layers.DiagExpectationUnitLayer(5,3, phase=1j))
M.add(layers.DiagExpectationUnitLayer(3,1, phase=1j))

In [ ]:
minim = minimizer.CMA(False)
minim.train(mse(), M, X, Y, maxiter=1000)

In [ ]:
npoints = 1000
test_X = (np.linspace(0, 1000, npoints)).reshape((1, npoints))

plt.plot(X.flatten(), Y.flatten(),"og", label='train')
plt.plot(test_X.flatten(), np.real(M.predict(test_X)).flatten(),"b-", label='fit', linewidth=5)
plt.legend()
plt.xlabel("t")
plt.ylabel("y")

# Example 3: Saw-tooth

In [ ]:
def func(x,n):
    return signal.sawtooth(x)

X = np.linspace(0, 30, 500)
X = X.reshape((1,X.shape[0]))

Y = func(X[:,None],1).reshape((1,X.shape[1]))

In [ ]:
plt.plot(X.flatten(), Y.flatten(),"ob-")

In [ ]:
M = mdl.Model()

M.add(layers.DiagExpectationUnitLayer(1,10, Wmax=0.5,Qmax=1,param_bound=10, phase=1j))
M.add(layers.DiagExpectationUnitLayer(10,1, Wmax=0.5,Qmax=1,param_bound=10, phase=1j))

minim = minimizer.CMA(parallel=False)
minim.train(mse(), M, X, Y, maxiter=1000)

In [ ]:
npoints = 500
test_X = (np.linspace(0, 30, npoints)).reshape((1, npoints))

plt.plot(X.flatten(), Y.flatten(),"og-", label='target')
plt.plot(test_X.flatten(), np.real(M.predict(test_X)).flatten(),"ob-", label='fit')
#plt.savefig("sawtooth_reg.eps", format="eps")
plt.legend()

# Example 3: MNIST (via CMA)

In [2]:
# Load MNIST dataset
MNIST_train = pd.read_csv('~/data/mnist_train.csv', delimiter=",",header=None).values
MNIST_test  = pd.read_csv('~/data/mnist_test.csv', delimiter=",",header=None).values

# Prepare data (normalized onto [0,1])
Y_train = MNIST_train[0:10000,0]
X_train = MNIST_train[0:10000,1:]/255.0

Y_test = MNIST_test[:,0]
X_test = MNIST_test[:,1:]/255.0

enc = LabelBinarizer()
enc.fit(np.diag([1,1,1,1,1,1,1,1,1,1]))
enc.classes_ = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
T=enc.transform(Y_train).T


In [ ]:
M = mdl.Model()
M.add(layers.DiagExpectationUnitLayer(784,10))
M.add(layers.DiagExpectationUnitLayer(10,10))
M.add(layers.DiagExpectationUnitLayer(10,10))
M.add(layers.SoftMaxLayer(10))

In [ ]:
minim = minimizer.CMA(False)

sol=minim.train(mse(), M, np.transpose(X_train), T, maxiter=10)

In [ ]:
P=np.argmax(M.predict(np.transpose(X_train)),axis=0)

print(classification_report(Y_train,P.T))
print(confusion_matrix(Y_train, P.T))

In [ ]:
P=np.argmax(M.predict(np.transpose(X_test)),axis=0)


print(classification_report(Y_test,P.T))
print(confusion_matrix(Y_test, P.T))

In [ ]:
M.get_layer(1).show_activation(9,1)

# Example 4: MNIST (via SGD)

In [20]:
M = mdl.Model()
M.add(layers.NonLinear(784,10))
M.add(layers.SoftMaxLayer(10))
#M.add(layers.DiagExpectationUnitLayer(784,10))
#

In [24]:
minim = minimizer.SGD()

sol=minim.train(mse(), M, np.transpose(X_train), T, lr=100, maxiter=1000)

Iteration 0 in 0.26(s), cost = 0.449289
Iteration 100 in 26.05(s), cost = 0.448915
Iteration 200 in 51.82(s), cost = 0.448705
Iteration 300 in 78.08(s), cost = 0.448510
Iteration 400 in 109.98(s), cost = 0.448325
Iteration 500 in 141.13(s), cost = 0.448142
Iteration 600 in 166.89(s), cost = 0.447955
Iteration 700 in 192.65(s), cost = 0.447765
Iteration 800 in 218.40(s), cost = 0.447574
Iteration 900 in 244.15(s), cost = 0.447385
('Cost: ', 0.44720197973814474)
('Sol: ', array([-0.52742672, -1.27278432, -0.28401372, ..., -0.28472256,
       -0.05574293,  0.35195976]))
Time: 273 s


In [ ]:
minim = minimizer.CMD(False)

sol=minim.train(mse(), M, np.transpose(X_train), T, maxiter=100)

In [22]:
P=np.argmax(M.predict(np.transpose(X_train)),axis=0)

print(classification_report(Y_train,P.T))
print(confusion_matrix(Y_train, P.T))

             precision    recall  f1-score   support

          0       0.02      0.02      0.02      1001
          1       0.01      0.01      0.01      1127
          2       0.12      0.06      0.08       991
          3       0.05      0.03      0.04      1032
          4       0.01      0.01      0.01       980
          5       0.12      0.06      0.08       863
          6       0.05      0.04      0.04      1014
          7       0.17      0.63      0.27      1070
          8       0.13      0.15      0.14       944
          9       0.10      0.07      0.08       978

avg / total       0.08      0.11      0.08     10000

[[ 18 178 210   0  94  88  52 182  93  86]
 [283   8   5  37 112  14 156 318 165  29]
 [ 24  53  59 109  15  35 221 308  80  87]
 [ 23  57  11  31 133  19  26 526 104 102]
 [ 58 129  30  77   5  39   9 540  48  45]
 [ 44 148  49  27  96  49  48 161 164  77]
 [ 24  43  15 218   5  12  40 296 290  71]
 [ 88  45  33   7  36  97  33 673   3  55]
 [112  29  67  73

In [23]:
P=np.argmax(M.predict(np.transpose(X_test)),axis=0)


print(classification_report(Y_test,P.T))
print(confusion_matrix(Y_test, P.T))

             precision    recall  f1-score   support

          0       0.04      0.03      0.03       980
          1       0.01      0.00      0.00      1135
          2       0.11      0.06      0.08      1032
          3       0.05      0.03      0.04      1010
          4       0.01      0.00      0.01       982
          5       0.13      0.05      0.07       892
          6       0.02      0.02      0.02       958
          7       0.17      0.65      0.27      1028
          8       0.10      0.11      0.10       974
          9       0.08      0.06      0.07      1009

avg / total       0.07      0.10      0.07     10000

[[ 26 144 238   1  90  64  70 160  95  92]
 [268   4   1  36  87   8 189 309 196  37]
 [ 54  39  58 129  14  38 245 270  83 102]
 [ 26  30  12  29 155  17  31 476  88 146]
 [ 56 149  23  78   4  41  16 536  34  45]
 [ 65  96  60  35 104  47  60 135 201  89]
 [ 42  25  17 184  11  16  21 312 263  67]
 [ 60  28  35  13  35  86  41 669  10  51]
 [110  26  58  87